# Data Preprocessing

After the exploration of the dataset, we will now preprocess the data. This includes:
- Removing unnecessary columns
- Handling outliers
- Encoding categorical variables
- Normalizing numerical variables

In this notebook, we will try building the preprocessing pipeline before extract to a separate module.

In [1]:
import pandas as pd

from ds_salaries_predict.config import RAW_DATA_DIR

data = pd.read_csv(RAW_DATA_DIR / "salaries.csv")

2025-05-23 00:29:05.659 | INFO     | ds_salaries_predict.config:<module>:11 - PROJ_ROOT path is: E:\Projects\ds-salaries-predict
